# Preparing

In [ ]:
import numpy as np
from tqdm import tqdm

import tensorflow as tf
import keras
from keras.preprocessing import timeseries_dataset_from_array
from keras.callbacks import CSVLogger
from sklearn.metrics import classification_report

import data
from modules.utilites import DataClass

seq_len = 100

In [ ]:
# Load data
row_data = (
    # data.load_dataset(horizon=4)
    data.load_saved_datas())
data.inspect_datas(row_data)

In [ ]:
from models.m_base import blocks

In [ ]:
# Model parametrs
class CN_default(DataClass):
    n_filters = 14
    dilation_steps = 4  # dilation = 2**dilation_step


class AN_default(DataClass):
    attention_heads = 3
    blocks = 2
    share_weights = False


class FF_default(DataClass):
    dropout_rate = 0.1


class Optimaser_default(DataClass):
    lr = 0.0001
    adam_beta1 = 0.9
    adam_beta2 = 0.999


class Trainig_default(DataClass):
    shuffle = True
    batch_size = 512
    epochs = 150


class Full_default(DataClass):
    cn = CN_default()
    an = AN_default()
    ff = FF_default()
    optimizer = keras.optimizers.Adam(
        learning_rate=0.0001,
        beta_1=0.9,
        beta_2=0.999,
        name="Adam",
    )
    training = Trainig_default()


default_pars = Full_default()

In [ ]:
# Datasets
datasets = data.build_datasets(
    datas=row_data,
    batch_size=default_pars.training.batch_size,
    seq_len=seq_len,
)
ds_train, ds_val, ds_test = datasets['train'], datasets['val'], datasets[
    'test']
data.inspect_datasets(datasets)

In [ ]:
# Model
inputs = blocks.input_block(seq_len)
x = inputs

x = blocks.cnn_block(
    input_layer=x,
    filters=default_pars.cn.n_filters,
    dilation_steps=default_pars.cn.dilation_steps,
)
x = blocks.norm_block(input_layer=x)
x = blocks.positional_encoder_block(input_layer=x)
x = blocks.transformer_block(
    input_layer=x,
    n_blocks=default_pars.an.blocks,
    n_heads=default_pars.an.attention_heads,
    share_weights=default_pars.an.share_weights,
)
x = blocks.ffn_block(
    input_layer=x,
    dropout_rate=default_pars.ff.dropout_rate,
)

model = keras.Model(inputs=inputs, outputs=x)
model.summary(
    line_length=80,
    expand_nested=True,
)

In [ ]:
# CompiLe
model.compile(
    optimizer=default_pars.optimizer,
    loss=keras.losses.SparseCategoricalCrossentropy(),
    metrics=[
        keras.metrics.CategoricalAccuracy(name='accurancy'),
        keras.metrics.SparseCategoricalAccuracy(name='sparce_accurancy'),
    ],
)

In [ ]:
# Train
model.fit(
    ds_train,
    epochs=default_pars.training.epochs,
    validation_data=ds_val,
    # callbacks=[
    #     CSVLogger(r'logs/model1/log.csv', append=True, separator=';')
    # ]
)